import necessary libaries

In [1]:
import pandas as pd

In [2]:
input_text = "AI_in_Education.txt"


In [3]:
with open(input_text, 'r', encoding='utf-8') as file:
    lines = file.readlines()
    content = file.read()


lines[1:5]

['\n',
 '[Applause]\n',
 'Who was your best\n',
 'professor? Not your favorite, your best,\n']

In [4]:
from pathlib import Path

# adjust path if your notebook is in the Dislexia folder already
txt_path = Path("AI_in_Education.txt")
if not txt_path.exists():
    txt_path = Path("Dislexia/AI_in_Education.txt")

with txt_path.open("r", encoding="utf-8") as f:
    text = f.read()

# quick check
print(len(text), "characters loaded")
print(text[:500])  # preview first 500 chars

11309 characters loaded
Title: AI Will Reshape Education. Are We Building Tools We Can Trust? | Jim Chilton | TEDxSNHU

[Applause]
Who was your best
professor? Not your favorite, your best,
the person you learned something from
that's still with you
today. Maybe it was something about
history that helped you understand the
past or a science experiment that helped
you understand how the world
worked. And then imagine this morning
you discovered that was
wrong. This is the quiet risk that is
happening across education
to


In [5]:
import os
from pathlib import Path
from dotenv import load_dotenv

# load .env from the notebook folder (Dislexia)
env_path = Path.cwd() / ".env"
if not env_path.exists():
    # try repository folder explicitly
    env_path = Path(__file__).parent if "__file__" in globals() else Path(".") 
    env_path = env_path / ".env"

if env_path.exists():
    load_dotenv(env_path)
else:
    # fallback: rely on environment variables already set
    pass


In [6]:
OPENROUTER_KEY = os.environ.get("OPENROUTER_API_KEY")
if not OPENROUTER_KEY:
    raise RuntimeError("Set OPENROUTER_API_KEY in your environment")

In [7]:
import requests
import json 

text_snippet = text if len(text) <= 3000 else text[:3000] + "\n\n...[truncated]"

prompt = (
    "Generate EXACTLY 5 multiple-choice questions (4 choices A-D) based ONLY on the source text. "
    "Return only valid JSON: a list of objects like "
    '{"question":"...","choices":["A ...","B ...","C ...","D ..."],"answer":"B","explanation":"..."}'
    f"\n\nSource text:\n\n{text_snippet}"
)


In [8]:
text_snippet = text_snippet[:200]
text_snippet

'Title: AI Will Reshape Education. Are We Building Tools We Can Trust? | Jim Chilton | TEDxSNHU\n\n[Applause]\nWho was your best\nprofessor? Not your favorite, your best,\nthe person you learned something f'

In [12]:
import requests, json, os

OPENROUTER_KEY = os.environ.get("OPENROUTER_API_KEY")

text_snippet = text if len(text) <= 3000 else text[:3000] + "\n\n...[truncated]"

prompt = (
    "Generate EXACTLY 3 multiple-choice questions (4 choices A-D) based ONLY on the source text. "
    "Return only valid JSON array: "
    '[{"question":"...","choices":["A ...","B ...","C ...","D ..."],"answer":"B","explanation":"..."}]'
    f"\n\nSource text:\n\n{text_snippet}"
)

resp = requests.post(
    "https://openrouter.ai/api/v1/chat/completions",
    headers={
        "Authorization": f"Bearer {OPENROUTER_KEY}",
        "Content-Type": "application/json",
    },
    json={
        "model": "openai/gpt-3.5-turbo",
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0.3,
        "max_tokens": 800,
    },
    timeout=30,
)

resp.raise_for_status()
content = resp.json()["choices"][0]["message"]["content"]

try:
    questions = json.loads(content)
    print(json.dumps(questions, indent=2))
except Exception as e:
    print(f"Parse error: {e}")
    print("Raw output:", content)

[
  {
    "question": "What are the traditional components of education mentioned in the text?",
    "choices": [
      "A. Computers, tablets, and smartphones",
      "B. Textbooks, teachers, and libraries",
      "C. Online courses, webinars, and podcasts",
      "D. Laboratories, research papers, and exams"
    ],
    "answer": "B",
    "explanation": "The text mentions textbooks, teachers, and libraries as the traditional components of education."
  },
  {
    "question": "What is one of the challenges with generative AI in education according to the text?",
    "choices": [
      "A. Lack of student engagement",
      "B. Lack of accountability and bibliography",
      "C. Slow response time",
      "D. Limited access to information"
    ],
    "answer": "B",
    "explanation": "One of the challenges with generative AI mentioned in the text is the lack of accountability and bibliography."
  },
  {
    "question": "How have students shifted their learning in today's education syste